# Apply global signal correction

Here, global signal correction is applied via [post-hoc mean centering](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4074397/) (after GLM)

In [1]:
import nilearn
from nilearn import image
import numpy as np
from nilearn import masking
import glob

# Function for subtracting global signal from one image

* Load in beta series image (each of the 24 volumes is the contrast parameter estimate map for a particular trial).
* Mask the image to just contain brain voxels
* Get the timeseries of the mean signal across the brain for each of the 24 volumes here, then subtract this timeseries from all voxels in each volume
* Zero out non-brain voxels and save images out


In [24]:
def subtractGlobalSignal(maskFile, imgFile):
    name = imgFile.split('/')[1]
    # load in 4D image image and 3d smask
    img = image.load_img(imgFile)
    mask = image.load_img(maskFile)
    
    # mask the 4d image
    sigMask = nilearn.masking.apply_mask(img, mask)
    
    # Get mean timeseries --- should be 24 elements
    meanTS = np.mean(sigMask, axis = 1)
    
    # subtract out the mean timeseries
    signal= img.get_data()
    maskOnes = mask.get_data()
    subTract = signal - meanTS
    
    # zero out other voxels
    a = np.transpose(subTract[maskOnes == 1])
    unmaskedImg = nilearn.masking.unmask(a, mask)
    unmaskedImg.to_filename('betaSeriesCopesGlobalSignalSubtract/' + name)

# Do this for all beta series cope images

In [8]:
images = glob.glob('betaSeriesCopesReg/*')

In [5]:
for cope in list(images):
    subtractGlobalSignal(maskFile = '../Structural/mni_2mm_brain_mask.nii.gz', imgFile = cope)